In [1]:
from gen import generate_vector
import json
import database_manager as dm
from pymilvus import FieldSchema, DataType, CollectionSchema

In [2]:
import importlib
importlib.reload(dm)

<module 'database_manager' from 'd:\\GPST\\mywork\\trymilvus\\database_manager.py'>

In [3]:
uri = "http://localhost:19530"
db_name = "test_db"
collection_name = "test_col"
dim = 16


In [4]:
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="primary id")
# extra_info = FieldSchema(name="ex_info", dtype=DataType.VARCHAR, description="extra information", max_length=10)
vector_field = FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=dim, description="vector")
# fields = [id_field, extra_info, vector_field]
fields = [id_field, vector_field]
schema = CollectionSchema(fields=fields)

In [5]:
client = dm.initialize_client(uri=uri, db_name=db_name)


In [7]:

collection = dm.initialize_collection(client=client, collection_name=collection_name, schema=schema)
dm.create_scalar_index(client=client, collection_name=collection_name)
dm.create_vector_index(client=client, collection_name=collection_name, field_name="vector")

In [8]:
print(json.dumps(client.describe_collection(collection_name=collection_name), indent=2))


{
  "collection_name": "test_col",
  "auto_id": false,
  "num_shards": 1,
  "description": "",
  "fields": [
    {
      "field_id": 100,
      "name": "id",
      "description": "primary id",
      "type": 5,
      "params": {},
      "is_primary": true
    },
    {
      "field_id": 101,
      "name": "vector",
      "description": "vector",
      "type": 101,
      "params": {
        "dim": 16
      }
    }
  ],
  "aliases": [],
  "collection_id": 449685445882160999,
  "consistency_level": 2,
  "properties": {},
  "num_partitions": 1,
  "enable_dynamic_field": false
}


In [9]:
print(client.list_indexes(collection_name=collection_name))
for index_name in client.list_indexes(collection_name=collection_name):
    print(json.dumps(client.describe_index(collection_name=collection_name, index_name=index_name), indent=2))
    

['scalar_index', 'vector_index']
{
  "index_type": "STL_SORT",
  "field_name": "id",
  "index_name": "scalar_index"
}
{
  "index_type": "FLAT",
  "metric_type": "L2",
  "field_name": "vector",
  "index_name": "vector_index"
}


In [10]:
vectors = generate_vector(cnt=5, dim=dim, mx=5)
data = dm.create_entries(vectors=vectors)
client.upsert(collection_name=collection_name, data=data)


{'upsert_count': 5}

In [11]:
client.load_collection(collection_name=collection_name)
print(client.query(collection_name=collection_name, filter="3<=id<=10",output_fields=["id", "vector"]))


[{'id': 3, 'vector': [0.7440138, -1.5468471, -1.3815701, -1.7754685, -4.4554243, 0.28276268, 0.7214891, -1.8314956, -0.05402502, 1.3735344, 1.2580377, -0.124183826, -3.5716295, -2.3025086, -0.8880386, 2.913734]}, {'id': 4, 'vector': [0.13458638, 3.1460488, 0.92004573, -0.3792929, -0.5139929, -3.9752426, 3.368728, -0.84479517, 4.269142, 4.3546596, 0.92113, 4.4358935, 4.4226913, 2.157279, 3.8224192, -1.0377874]}, {'id': 5, 'vector': [-2.5981934, 4.6048746, 0.18097398, -3.6399827, 3.7097256, 2.1265006, 1.2909718, -0.43676138, -3.0506036, -3.3720016, -4.326305, 1.102544, -1.2360549, 2.3676863, 0.36953095, 3.0712276]}]


In [12]:
vectors = generate_vector(cnt=5, dim=dim, mx=5)
data = dm.create_entries(vectors=vectors, ids=[2,3,4,5,6])
client.upsert(collection_name=collection_name, data=data)

{'upsert_count': 5}

In [13]:
client.load_collection(collection_name=collection_name)
print(client.query(collection_name=collection_name, filter="3<=id<=10",output_fields=["id", "vector"]))


[{'id': 3, 'vector': [-2.4593275, -4.536549, 3.9471092, 3.0760763, -2.9431765, -4.567603, 3.637244, -0.6058879, 3.693978, 4.341534, 1.3014174, 2.0279787, -0.1514137, -1.7452315, 0.32339856, 1.9948853]}, {'id': 4, 'vector': [-0.009040037, -4.3773456, -1.9189106, -3.6090865, 0.7528663, -3.5869615, 0.83341104, -3.8600447, 2.1926188, 4.604464, 2.3016715, 1.4256666, 4.4402337, 1.2564477, -3.194402, -1.5945774]}, {'id': 5, 'vector': [-3.3658519, 2.385287, -3.3778691, -1.3815538, -3.3658893, 2.6481626, 0.7066092, 2.3477223, -3.2879288, 1.7602117, 4.143479, -4.2692857, -1.187632, 2.4332628, 0.16690332, -1.6893855]}, {'id': 6, 'vector': [2.1865954, 0.5424491, -1.7929579, 4.090019, 2.782146, -0.048798874, -2.4762669, -0.24866274, -4.50565, 4.261579, 2.6250494, -0.6355131, -4.4113493, 4.9396825, 3.7217698, -1.9897586]}]
